In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
import requests
import time
import datetime as dt


from dotenv import load_dotenv
from flipside import Flipside
from prophet import Prophet

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# from sql_queries.sql_scripts import eth_price

In [2]:
Optimistic_Domains_Sales_query = """
  SELECT
      DATE_TRUNC('HOUR', BLOCK_TIMESTAMP) AS day, tokenid, price
    FROM
      optimism.nft.ez_nft_sales
    WHERE
      NFT_ADDRESS = LOWER('0xC16aCAdf99E4540E6f4E6Da816fd6D2A2C6E1d4F')
      AND event_type = 'sale'

"""

eth_price = """

  select
    hour as dt,
    symbol,
    price
  from
    ethereum.price.ez_prices_hourly
  where
    symbol in('WETH')
    AND date_trunc('day', dt) >= '	2022-06-01'
  order by
    dt DESC

"""

In [3]:
os.chdir('..')
print(os.getcwd())

e:\Projects\liquid_domains


In [4]:
from scripts.utils import flipside_api_results, set_random_seed
from scripts.pull_data import pull_data
from scripts.data_processing import process_data

In [5]:
os.chdir('Notebooks')

In [6]:
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
current_directory = os.getcwd()
current_directory

'e:\\Projects\\liquid_domains\\Notebooks'

In [8]:
load_dotenv()

True

In [9]:
seed = 20
set_random_seed(seed)

In [10]:
flipside_api_key = os.getenv('FLIPSIDE_API_KEY')
alchemy_api_key = os.getenv('ALCHEMY_API_KEY')
opensea_api_key = os.getenv('OPENSEA_API_KEY')

In [11]:
api = True #Pulls the most recent data from all web3 sources, web2 sources are still via csv (This function takes 2+ hours, use it periodically)
web2_data = True #Includes web2 data in training
threshold = None #Correlation value for correlation analysis
temporals = True #Tends to inflate estimation
fine_tuning_web3 = True #Train again on web3 data only
correlation_analysis = False #Performs Correlation Analysis to 'price' col and drops cols under the threshold parameter 
last_dataset = False #Saves time of feature engineering, saves the latest copy of the dataset generated by the function

# To preview web2 domain data; Kaggle dataset and namebio dataset

domain_path = '../data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')

domain_data.set_index('date', inplace=True)
domain_data = domain_data.drop(columns=['venue'])
domain_data.sort_index(inplace=True)
domain_data

domain_data = domain_data.reset_index()
domain_data = domain_data.rename(columns={"date":"dt","price":"price_usd"})
domain_data['dt'] = pd.to_datetime(domain_data['dt'])
domain_data['dt'] = domain_data['dt'].dt.tz_localize('UTC')
domain_data['dt'] = pd.to_datetime(domain_data['dt'])

domain_data['web3'] = False

namebio_path = '../data/namebio_sales.csv'
namebio_data = pd.read_csv(namebio_path)
namebio_data.set_index('Date', inplace=True)
namebio_data = namebio_data.drop(columns=['Venue'])
namebio_data.sort_index(inplace=True)

namebio_data = namebio_data.reset_index()
namebio_data = namebio_data.rename(columns={"Date":"dt","Price":"price_usd","Domain":"domain"})
namebio_data['dt'] = pd.to_datetime(namebio_data['dt'])
namebio_data['dt'] = namebio_data['dt'].dt.tz_localize('UTC')
namebio_data['dt'] = pd.to_datetime(namebio_data['dt'])

namebio_data['web3'] = False

domain_data = pd.concat([domain_data,namebio_data],ignore_index=True)

In [12]:
X, y, prophet_features, gen_features, target, combined_dataset, features, web3_data, X_web3, y_web3 = process_data(api=api,web2_data=web2_data,threshold=threshold,temporals=temporals,correlation_analysis=correlation_analysis,last_dataset=last_dataset)

starting process_data
6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5
Pulling Fresh Data...
Fetching sale: Page 1, Events Fetched: 50, Total Events: 50, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTEyLTMwKzA3JTNBMjAlM0EyMyYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0zMDQ3MjYzMjU1OQ==
Fetching sale: Page 2, Events Fetched: 50, Total Events: 100, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTEyLTI5KzE5JTNBMjglM0E1OSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0zMDQ1NDc1MDU5Nw==
Fetching sale: Page 3, Events Fetched: 50, Total Events: 150, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTEyLTI3KzE5JTNBNDUlM0EzNSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0zMDM4NTQ5MDAzOQ==
Fetching sale: Page 4, Events Fetched: 50, Total Events: 200, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTEyLTI2KzEyJTNBMDglM0EyMyYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0zMDMzODYyNzk0MA==
Fetching sale: Page 5, Events Fetched: 50, Total Events: 250, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTEyLTI1KzA4JTNBNDAlM0EzNSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0zMDI4NTcwMTU1NA==
Fetc

AttributeError: 'NoneType' object has no attribute 'query'

In [13]:
combined_dataset['price_usd'].max()

np.float64(30000000.0)

In [14]:
combined_dataset[combined_dataset['word_count']>1]

,Unnamed: 0,dt,domain,price_usd,web3,domain_length,num_vowels,num_consonants,tld,word_count,...,tld_length,is_brandable,levenshtein_distance,is_subdomain,domain_entropy,rank,tld_weight,7d_rolling_avg_price,7d_sales_volume,7d_rolling_std_dev
228,228,2015-07-08 00:00:00+00:00,interfaith-center.org,110.00,0,21,7,12,org,2,...,3,1,8,0,3.52,unranked,1,"1,451.29","10,159.00","3,272.02"
407,407,2015-07-14 00:00:00+00:00,face-europe.org,490.00,0,15,7,6,org,2,...,3,0,5,0,3.32,unranked,1,332.71,"2,329.00",151.97
463,463,2015-07-18 00:00:00+00:00,velo-city2014.com,185.00,0,17,4,7,com,2,...,3,0,8,0,3.85,unranked,1,612.00,"4,284.00",550.70
554,554,2015-07-26 00:00:00+00:00,outer-rim.net,120.00,0,13,5,6,net,2,...,3,0,4,0,3.24,unranked,1,204.29,"1,430.00",141.84
669,669,2015-07-26 00:00:00+00:00,festival-kinetik.net,206.00,0,20,7,11,net,2,...,3,0,9,0,3.41,unranked,1,222.00,"1,554.00",192.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565698,565698,2024-10-11 00:00:00+00:00,perdre-la-raison.com,305.00,0,20,7,10,com,3,...,3,1,9,0,3.68,unranked,1,"2,003.43","14,024.00",917.53
565703,565703,2024-10-11 05:00:00+00:00,step-brother.eth,1.69,1,16,4,10,eth,2,...,3,0,5,0,3.16,unranked,1,121.97,853.81,121.56
565715,565715,2024-10-11 17:00:00+00:00,v-bucks.eth,"3,665.09",1,11,2,7,eth,2,...,3,0,4,0,3.46,unranked,1,880.93,"6,166.48","1,348.77"
565721,565721,2024-10-12 00:00:00+00:00,re-captha-version-3-31.top,146.00,0,26,7,11,top,5,...,3,0,14,0,3.85,unranked,1,119.43,836.00,23.00


In [15]:
combined_dataset[combined_dataset['rank']!='unranked']

,Unnamed: 0,dt,domain,price_usd,web3,domain_length,num_vowels,num_consonants,tld,word_count,...,tld_length,is_brandable,levenshtein_distance,is_subdomain,domain_entropy,rank,tld_weight,7d_rolling_avg_price,7d_sales_volume,7d_rolling_std_dev
0,0,1999-04-01 00:00:00+00:00,bingo.com,"1,100,000.00",0,9,3,5,com,1,...,3,0,1,0,2.95,75007,1,"622,779.22","4,359,454.53","1,228,655.56"
1,1,1999-04-01 00:00:00+00:00,altavista.com,"3,250,000.00",0,13,5,7,com,1,...,3,0,0,0,3.18,6188,1,"465,643.35","3,259,503.47","1,227,787.88"
2,2,1999-11-01 00:00:00+00:00,fly.com,"1,500,000.00",0,7,1,5,com,1,...,3,0,2,0,2.81,903841,1,"836,411.67","5,854,881.72","1,233,249.92"
3,3,1999-12-01 00:00:00+00:00,tom.com,"2,500,000.00",0,7,2,4,com,1,...,3,0,1,0,2.24,15127,1,"1,793,551.35","12,554,859.42","1,048,390.72"
5,5,1999-12-01 00:00:00+00:00,autos.com,"2,200,000.00",0,9,4,4,com,1,...,3,0,2,0,2.95,852369,1,"1,150,697.37","8,054,881.62","1,264,502.04"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565120,565120,2024-10-10 00:00:00+00:00,clomida.com,460.00,0,11,4,6,com,1,...,3,0,3,0,2.91,664783,1,480.57,"3,364.00",21.05
565245,565245,2024-10-10 00:00:00+00:00,realtime.co,"3,383.00",0,11,5,5,co,1,...,2,0,2,0,3.28,479337,1,"2,581.57","18,071.00",623.57
565260,565260,2024-10-10 00:00:00+00:00,circleof6app.com,"2,309.00",0,16,5,9,com,1,...,3,0,6,0,3.45,684382,1,"2,507.00","17,549.00",289.28
565439,565439,2024-10-11 00:00:00+00:00,shoeswholesalesuppliers.us,205.00,0,26,10,15,us,1,...,2,0,14,0,3.31,663874,1,185.14,"1,296.00",17.05


domain_rankings = pd.read_csv('../data/tranco_5863N.csv')
google_rank = pd.DataFrame({'rank': [1], 'domain': ['google.com']})
domain_rankings.columns = ['rank','domain']

# Concatenate the new row with the original domain rankings
domain_rankings = pd.concat([google_rank, domain_rankings], ignore_index=True)

# Reset the index and display the updated rankings
domain_rankings.reset_index(drop=True, inplace=True)

domain_rankings['domain'].head(500).values

# Sales

In [16]:
from models.model import train_model

In [17]:
combined_dataset

,Unnamed: 0,dt,domain,price_usd,web3,domain_length,num_vowels,num_consonants,tld,word_count,...,tld_length,is_brandable,levenshtein_distance,is_subdomain,domain_entropy,rank,tld_weight,7d_rolling_avg_price,7d_sales_volume,7d_rolling_std_dev
0,0,1999-04-01 00:00:00+00:00,bingo.com,"1,100,000.00",0,9,3,5,com,1,...,3,0,1,0,2.95,75007,1,"622,779.22","4,359,454.53","1,228,655.56"
1,1,1999-04-01 00:00:00+00:00,altavista.com,"3,250,000.00",0,13,5,7,com,1,...,3,0,0,0,3.18,6188,1,"465,643.35","3,259,503.47","1,227,787.88"
2,2,1999-11-01 00:00:00+00:00,fly.com,"1,500,000.00",0,7,1,5,com,1,...,3,0,2,0,2.81,903841,1,"836,411.67","5,854,881.72","1,233,249.92"
3,3,1999-12-01 00:00:00+00:00,tom.com,"2,500,000.00",0,7,2,4,com,1,...,3,0,1,0,2.24,15127,1,"1,793,551.35","12,554,859.42","1,048,390.72"
4,4,1999-12-01 00:00:00+00:00,england.com,"2,000,000.00",0,11,3,7,com,1,...,3,0,2,0,3.28,unranked,1,"1,436,408.50","10,054,859.52","1,184,598.24"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565729,565729,2024-10-12 00:00:00+00:00,revenueserver.com,105.00,0,17,7,9,com,1,...,3,0,5,0,3.01,unranked,1,288.14,"2,017.00",209.08
565730,565730,2024-10-12 00:00:00+00:00,alt-minds.com,105.00,0,13,3,8,com,2,...,3,0,4,0,3.55,unranked,1,227.71,"1,594.00",188.30
565731,565731,2024-10-12 00:00:00+00:00,amazingusambaratrekking.com,100.00,0,27,10,16,com,1,...,3,0,14,0,3.78,unranked,1,170.00,"1,190.00",146.86
565732,565732,2024-10-12 00:00:00+00:00,👨‍⚕️.eth ⚠️,"4,859.42",1,11,1,2,eth ⚠️,1,...,6,0,4,0,3.28,unranked,1,"1,881.22","13,168.55","2,351.85"


In [18]:
results = train_model(X, y, prophet_features, gen_features, target, combined_dataset, features, web3_data, X_web3, y_web3, seed=seed, web3=fine_tuning_web3)

prophet features: Index(['7d_rolling_avg_price', '7d_rolling_std_dev', '7d_sales_volume',
       'Unnamed: 0', 'domain', 'domain_entropy', 'domain_length',
       'has_numbers', 'is_brandable', 'is_subdomain', 'levenshtein_distance',
       'num_consonants', 'num_vowels', 'rank', 'tld', 'tld_length',
       'tld_weight', 'web3', 'word_count'],
      dtype='object')
Training on Combined Data...
Index(['Unnamed: 0', 'dt', 'domain', 'price_usd', 'web3', 'domain_length',
       'num_vowels', 'num_consonants', 'tld', 'word_count', 'has_numbers',
       'tld_length', 'is_brandable', 'levenshtein_distance', 'is_subdomain',
       'domain_entropy', 'rank', 'tld_weight', '7d_rolling_avg_price',
       '7d_sales_volume', '7d_rolling_std_dev'],
      dtype='object')


16:01:28 - cmdstanpy - INFO - Chain [1] start processing
16:06:04 - cmdstanpy - INFO - Chain [1] done processing


MemoryError: Unable to allocate 863. MiB for an array with shape (1000, 113093) and data type float64

In [ ]:
# os.chdir('..')
print(os.getcwd())

os.chdir('data')
print(os.getcwd())

# Valuation Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib

In [ ]:
prophet_model = joblib.load('../pkl/prophet_model.pkl')
ridge_model = joblib.load('../pkl/ridge_model.pkl')
randomforest_model = joblib.load('../pkl/randomforest_model.pkl')

In [ ]:
from models.forecasters import Prophet_Domain_Valuator, Domain_Valuator

In [ ]:
prophet_features

In [ ]:
combined_dataset['dt']

In [ ]:
def main(domain, prophet_model, ridge_model, randomforest_model, combined_dataset, prophet_features, gen_features, features, X, y,seed=seed,ensamble_method = 'median'):
    set_random_seed(seed)

    prophet_features_data = combined_dataset.copy()
    prophet_features_data.rename(columns={"dt": "ds", "price_usd": "y"}, inplace=True)

    prophet_valuator = Prophet_Domain_Valuator(domain, prophet_features, prophet_features_data)
    prophet_valuator.model_prep()
    prophet_domain_value = prophet_valuator.value_domain(prophet_model)

    features_data = combined_dataset.copy()
    features_data['dt'] = features_data['dt'].dt.tz_localize(None)
    features_data = features_data[features] 

    ridge_valuator = Domain_Valuator(domain, X, y, gen_features, features_data, seed)
    ridge_valuator.model_prep()
    ridge_domain_value = ridge_valuator.value_domain(ridge_model)

    randomforest_valuator = Domain_Valuator(domain, X, y, gen_features, features_data, seed)
    randomforest_valuator.model_prep()
    randomforest_domain_value = randomforest_valuator.value_domain(randomforest_model)

    individual_predictions = [
        prophet_domain_value,
        ridge_domain_value,
        randomforest_domain_value
    ]

    if ensamble_method == 'mean':
        ensemble_domain_value = np.mean(individual_predictions)
    elif ensamble_method == 'median':
        ensemble_domain_value = np.median(individual_predictions)

    print(f'individual valuations: {individual_predictions}')
    print(f'ensamble value: {ensemble_domain_value}')

    if ensemble_domain_value < 0:
        print(f'Defaulting from Negative to $0')
        ensemble_domain_value = 0


    return ensemble_domain_value

In [ ]:
domain = 'google.com'

In [ ]:
combined_dataset[combined_dataset['is_brandable']==1]

In [ ]:
value = main(domain=domain, prophet_model=prophet_model, ridge_model=ridge_model, randomforest_model=randomforest_model, combined_dataset=combined_dataset, prophet_features=prophet_features, gen_features=gen_features, features=features, X=X, y=y,ensamble_method='mean')

from sklearn.model_selection import GridSearchCV

# Define TLD weight grid
param_grid = {
    'tld_weight': [{'com': w_com, 'net': w_net, 'org': w_org, 'eth': w_eth}
                   for w_com in [1.0, 1.5, 2.0]
                   for w_net in [0.8, 1.2, 1.5]
                   for w_org in [1.0, 1.5, 2.0]
                   for w_eth in [1.0, 2.0, 3.0]]
}

grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, cv=5)
grid_search.fit(X, y)
best_weights = grid_search.best_params_


In [ ]:
params = {
    "New API Data":api,
    "Used web2 Data":web2_data,
    "Used threshold for correlation value":threshold,
    "Used temporals":temporals,
    "Fine tuned on web3 data":fine_tuning_web3,
    "Domain":domain,
    "Ensamble Value":value
}

params_df = pd.DataFrame([params])

In [ ]:
today = dt.datetime.today().strftime("%d-%m-%y-%H-%M-%S")

today = str(today)

# results_df = pd.DataFrame([results])

# combined_df = pd.concat([results_df, params_df], axis=1)

combined_df = params_df.copy()

combined_df.to_csv(f'../data/results_{today}.csv')

print(combined_df)

In [ ]:
liquidity_discount = 0.3 #If we want to discount 90%, we only take 10% of value 

discounted_value = value * liquidity_discount

liquidity_discount_filtered = 1 - liquidity_discount

print(f'Domain: {domain} \nValue: ${value:,.2f} \nLiquidity Discount: {liquidity_discount_filtered*100:.2f}% \nDiscounted Price w/ Liquidity Discount: ${discounted_value:,.2f}')

In [ ]:
subdomain_values = combined_dataset[combined_dataset['is_subdomain']==1]['domain'].unique()

In [ ]:
subdomain_struct = enumerate(subdomain_values)


for idx, val in enumerate(subdomain_values):
    print(f'val: {val}')

In [ ]:
combined_dataset['tld'].unique()

In [ ]:
def search(domain, tld, liquidity_discount=0.3,ensamble_method='median'):
    web3_tlds = ['.op', '.crypto', '.box', '.eth', '.base', '.nft', '.wallet', '.coin', '.finance', '.xyz', '.dao', '.bitcoin', '.blockchain']

    print(f'domain: {domain}')
    print(f'tld: {tld}')

    # Allow subdomains, so no error for dot in domain
    # Check if the TLD ends with any of the Web3 TLDs
    if not any(tld.endswith(web3_tld) for web3_tld in web3_tlds):
        raise ValueError('Error: Cannot Value Non-Web3 Domain')

    full_domain = domain + tld
    value = main(domain=full_domain, prophet_model=prophet_model, ridge_model=ridge_model, randomforest_model=randomforest_model, combined_dataset=combined_dataset, prophet_features=prophet_features, gen_features=gen_features, features=features, X=X, y=y,ensamble_method=ensamble_method)

    discounted_value = value * liquidity_discount
    liquidity_discount_filtered = 1 - liquidity_discount

    print(f'Domain: {full_domain} \nValue: ${value:,.2f} \nLiquidity Discount: {liquidity_discount_filtered*100:.2f}% \nDiscounted Price w/ Liquidity Discount: ${discounted_value:,.2f}')
    
    return value, discounted_value

    

In [ ]:
domain='uniswap'
tld='.crypto'

In [ ]:
search(domain,tld,liquidity_discount=0.5,ensamble_method='mean')

In [ ]:
op_domains = combined_dataset[combined_dataset['tld']=='op']
op_domains.to_csv('../data/op_domains_dataset.csv')

# combined_dataset[combined_dataset['tld']=='op']


for tld in combined_dataset['tld'].unique():
    print(f'TLD: {tld} \nShape: {combined_dataset[combined_dataset["tld"]==tld].shape}')

In [ ]:
combined_dataset

In [ ]:
num_of_tld = combined_dataset.groupby('tld')['domain'].count().sort_values(ascending = False)

num_of_tld.head(25)

num_of_tld[num_of_tld.index=='net']

In [ ]:
combined_dataset[combined_dataset['tld']=='op'].shape

In [ ]:
web3_data[web3_data.index == web3_data['price_usd'].idxmax()]

In [ ]:
combined_dataset[combined_dataset.index == combined_dataset['price_usd'].idxmax()]

In [ ]:
combined_dataset[combined_dataset['price_usd']==True].describe()

In [ ]:
combined_dataset[combined_dataset['web3']==True].describe()

In [ ]:
combined_dataset[combined_dataset['web3']==False].describe()

dcc.Input(
            id='valuator-input',
            value='example',
            type='text',
            style={
                'padding': '10px',
                'borderRadius': 'var(--wcm-input-border-radius)',
                'border': '1px solid var(--color-border)',
                'marginRight': '10px'
            },
            pattern='[^.]*'  # Regex pattern to disallow '.' character

# Things to show on front end
the metrics we already show
top 10 most sold domains 
top 10 most recent domains 
highest sold domains (top 5 or 10)
Time series for each top 10 most sold domain

Metrics by web3 == False or True


# Debugging

In [ ]:
Optimistic_Domains_Sales_query = """
  SELECT
  DATE_TRUNC('HOUR', BLOCK_TIMESTAMP) AS day,
  tokenid,
  price,
  price_usd
FROM
  optimism.nft.ez_nft_sales
WHERE
  NFT_ADDRESS = LOWER('0xC16aCAdf99E4540E6f4E6Da816fd6D2A2C6E1d4F')
  AND event_type = 'sale'
order by
  DATE_TRUNC('HOUR', BLOCK_TIMESTAMP) asc

"""

In [ ]:
optimistic_domains_sales_path = '../data/optimistic_domains_sales.csv'

In [ ]:
optimistic_domains_sales = flipside_api_results(Optimistic_Domains_Sales_query,flipside_api_key)
optimistic_domains_sales.to_csv(optimistic_domains_sales_path, index=False)

In [ ]:
from scripts.apis import alchemy_metadata_api

In [ ]:
eth_price = """

  select
    hour as dt,
    symbol,
    price
  from
    ethereum.price.ez_prices_hourly
  where
    symbol in('WETH')
    AND date_trunc('day', dt) >= '2019-12-01'
  order by
    dt DESC

"""

prices_path = '../data/prices.csv'
prices = flipside_api_results(eth_price, flipside_api_key)
prices.to_csv(prices_path,index=False)



Optimistic_domains_path = '../data/optimistic_domains_metadata.json'



optimistic_domains = alchemy_metadata_api(alchemy_api_key, 'optimism', '0xC16aCAdf99E4540E6f4E6Da816fd6D2A2C6E1d4F')
optimistic_domains.to_json(Optimistic_domains_path, orient='records')

In [ ]:
domain_rankings = pd.read_csv('../data/tranco_5863N.csv')
google_rank = pd.DataFrame({'rank': [1], 'domain': ['google.com']})
domain_rankings.columns = ['rank','domain']

# Concatenate the new row with the original domain rankings
domain_rankings = pd.concat([google_rank, domain_rankings], ignore_index=True)

# Reset the index and display the updated rankings
domain_rankings.reset_index(drop=True, inplace=True)

domain_rankings

In [ ]:
debug_data = pull_data(api=False)

In [ ]:
ens_debug = debug_data['ens']
ens_debug[ens_debug['nft_name']=='mhd.eth'].columns

In [ ]:
ens_debug[ens_debug.index == ens_debug['token_amt_clean'].idxmax()][['dt','token_symbol']]

In [ ]:
ens_debug['token_symbol'] = ens_debug['token_symbol'].str.lower()
ens_debug['token_symbol']

In [ ]:
debug_data['prices_data']

In [ ]:
debug_pd = debug_data['prices_data']

unique_dt = debug_pd['dt'].unique()

# Step 2: Create a new DataFrame with 'USDC' symbol and price 1 for each 'dt'
usdc_rows = pd.DataFrame({
    'dt': unique_dt,
    'symbol': 'USDC',     # Add 'USDC' as the symbol
    'price': 1.0          # Add 1 as the price
})

# Step 3: Concatenate the new rows with the existing DataFrame
debug_pd_combined = pd.concat([debug_pd, usdc_rows], ignore_index=True)

# Step 4: Optional - Sort by 'dt' and reset the row index
debug_pd_combined = debug_pd_combined.sort_values(by='dt').reset_index(drop=True)
debug_pd_combined.rename(columns={'symbol':'token_symbol'},inplace=True)

In [ ]:
debug_pd_combined['token_symbol'] = debug_pd_combined['token_symbol'].str.lower()
debug_pd_combined[['dt','token_symbol']]

In [ ]:
debug_pd_combined['dt'] = pd.to_datetime(debug_pd_combined['dt']).dt.strftime('%Y-%m-%d %H-00-00')
ens_debug['dt'] = pd.to_datetime(ens_debug['dt']).dt.strftime('%Y-%m-%d %H-00-00')

In [ ]:
ens_debug[['dt','token_symbol']]

In [ ]:
ens_debug['dt']

In [ ]:
test = pd.merge(debug_pd_combined,ens_debug,on=['dt','token_symbol'],how='right')
test['price_usd'] = test['token_amt_clean'] * test['price']
test[['nft_name','price_usd']]

In [ ]:
ens_debug[ens_debug['token_symbol']=='eth']